# Categorical Cross Entropy

## CCE Calculation

In [31]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt

batch_size, n_class = 16, 5

predictions = torch.rand(batch_size, n_class, dtype = torch.float32)
pred_sum = (torch.sum(predictions, 1)).view(-1, 1)
#print(predictions.shape, pred_sum.shape)

# 각각의 y^은 softmax를 통과하여 (0,1) 사이의 확률값을 가진다.
predictions = predictions / pred_sum

labels = torch.randint(0, n_class , (batch_size, ))
labels = labels.squeeze()

print(predictions.shape, labels.shape)

# nn.CrossEntropyLoss는 자체적으로 Softmax가 포함된 상태로 나오기 때문에 CCE만 적용했을 때와 결과가 다르게 나온다.
loss_object = nn.CrossEntropyLoss()
loss = loss_object(predictions, labels)

print(loss.numpy())

ce = 0
for label, prediction in zip(labels, predictions):
    ce += -torch.log(prediction[label]) 
    
ce /= batch_size
print(ce.numpy())

torch.Size([16, 5]) torch.Size([16])
1.6792479
2.4826763


## CCE with Model/Dataset

In [39]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt


N, n_features = 100, 2
n_class = 5

X = torch.zeros(0, n_features)
Y = torch.zeros(0, 1, dtype = torch.int32)


for class_idx in range(n_class):
    center = 15 * torch.clamp(torch.randn(2, 1), min = -1, max = 1) # coordinate values
    x1 = center[0] + torch.randn(N, 1)
    x2 = center[1] + torch.randn(N, 1)    
    
    x = torch.concat((x1, x2), axis = 1)
    y = class_idx * torch.ones(N, 1, dtype = torch.int32)
   
    X = torch.concat((X, x), axis = 0)
    Y = torch.concat((Y, y), axis = 0)
        

# 하나의 Dense Layer와 MSE를 계산하는 모델 설계
class Dense(torch.nn.Module):
    def __init__(self):
        super(Dense, self).__init__()
        self.layer = nn.Linear(2, 5)
        # 자체적으로 Softmax에 먼저 통과시킨다.
        self.cce = nn.CrossEntropyLoss()
        
        
    def forward(self, x):
        x = self.layer(x)
        #x = self.cce(x)
        return x
    
model = Dense()
loss_object = nn.CrossEntropyLoss()


# Dataset 클래스를 상속받아 커스텀 데이터셋 클래스 생성
class custom_dataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len
    
    
Y = Y.type(torch.FloatTensor)
Y = Y.squeeze()

# 인스터스(데이터) 생성
train_data = custom_dataset(X, Y)

print(X.shape, Y.shape)

# 만들어진 데이터가 배치형태로 만들어줘야하니까 DataLoader에다가 넣어줌
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)



for x, y in train_loader:
    predictions = model(x)
    y = y.type(torch.LongTensor)
    #type(predictions)
    loss = loss_object(predictions, y)
    print(loss.detach().numpy())

torch.Size([500, 2]) torch.Size([500])
1.1060883
3.5521147
3.5483196
5.533059
4.4997
1.9742541
5.1603546
3.7106035
1.1954383
2.9240706
5.6238446
4.1680145
5.5372324
2.8073113
6.2485104
4.7256484
4.6882586
4.7368364
2.793467
4.8260326
3.5557134
2.9427226
3.6391797
5.8403754
3.8241858
4.6248055
3.7496681
4.69248
4.3548446
3.403552
1.7836508
4.381799
